In [1]:
!pip install pandas
!pip install numpy
!pip install re
!pip install os
!pip install matplotlib
!pip install statsmodels
!pip install seaborn
!pip install scipy.stats
!pip install scikit_posthocs

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


ERROR: Could not find a version that satisfies the requirement scipy.stats (from versions: none)
ERROR: No matching distribution found for scipy.stats


In [2]:
'''PART 1: LINE PLOT GENERATION FOR ABSORBED PAR ON EACH RANK.'''

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import sem, t

# Load datasets for high and low densities
data_high = pd.read_csv('combined_files/combined_high_ranks_cleaned.csv')
data_low = pd.read_csv('combined_files/combined_low_ranks_cleaned.csv')

# Function to calculate mean, std dev, and confidence intervals
def calculate_statistics(data, confidence=0.95):
    grouped = data.groupby(['architecture', 'rank'])['absorbedPAR_umol_m2_s1']
    mean_values = grouped.mean().reset_index(name='mean')
    mean_values['std_dev'] = grouped.std().reset_index(name='std_dev')['std_dev']
    mean_values['n'] = grouped.count().reset_index(name='count')['count']
    
    # Calculate the t-value for 95% confidence interval
    mean_values['t_value'] = mean_values['n'].apply(lambda x: t.ppf((1 + confidence) / 2., x - 1))
    
    # Calculate the margin of error
    mean_values['margin_error'] = mean_values['t_value'] * (mean_values['std_dev'] / np.sqrt(mean_values['n']))
    
    # Calculate lower and upper confidence bounds
    mean_values['ci_lower'] = mean_values['mean'] - mean_values['margin_error']
    mean_values['ci_upper'] = mean_values['mean'] + mean_values['margin_error']
    
    return mean_values

# Calculate statistics for both high-density and low-density datasets
mean_values_high = calculate_statistics(data_high)
mean_values_low = calculate_statistics(data_low)

# Plotting function to avoid repetition
def plot_absorbedPAR(mean_values, density_label, output_file):
    plt.figure(figsize=(10, 6))
    
    # Plotting lineplot for each architecture type within the current density
    for architecture in mean_values['architecture'].unique():
        architecture_data = mean_values[mean_values['architecture'] == architecture]
        sns.lineplot(x='rank', y='mean', data=architecture_data, label=architecture, marker='o')
        
        # Adding the confidence interval as a shaded area
        plt.fill_between(architecture_data['rank'], architecture_data['ci_lower'], architecture_data['ci_upper'], alpha=0.3)
    
    plt.title(f'Absorbed PAR over Leaf Ranks for {density_label} Density')
    plt.xlabel('Rank')
    plt.ylabel('Absorbed PAR (umol m-2 s-1)')
    plt.ylim(0, 300)
    plt.xticks(range(1, 11))
    plt.legend(title='Architecture Type')
    plt.savefig(output_file)
    plt.close()

# Plot for high-density data
plot_absorbedPAR(mean_values_high, 'High', 'plots/mean_absorbedPAR_for_high_density.png')

# Plot for low-density data
plot_absorbedPAR(mean_values_low, 'Low', 'plots/mean_absorbedPAR_for_low_density.png')

print("Plots generated successfully for high and low densities.")

C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3

Kruskal-Wallis processing complete for all files.


C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")
C:\Users\jacop\AppData\Local\Temp\ipykernel_28516\3511155976.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  posthoc = posthoc.applymap(lambda x: f"{x:.3f}")


In [ ]:
'''PART 2: LINE PLOT FOR  RANKS of ABSORBED PAR OVER EACH LEAF RANK.'''

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import rankdata

# Load datasets for high and low densities
data_high = pd.read_csv('combined_files/combined_high_ranks_cleaned.csv')
data_low = pd.read_csv('combined_files/combined_low_ranks_cleaned.csv')

# Function to calculate ranks for absorbed PAR within each architecture and leaf rank
def calculate_ranks(data):
    # Group by architecture and rank, then calculate the rank for absorbed PAR
    data['rank_absorbedPAR'] = data.groupby(['architecture', 'rank'])['absorbedPAR_umol_m2_s1'].transform(lambda x: rankdata(x, method='average'))
    # Calculate the mean rank for each architecture and leaf rank
    mean_ranks = data.groupby(['architecture', 'rank'])['rank_absorbedPAR'].mean().reset_index(name='mean_rank')
    
    return mean_ranks

# Calculate ranks for both high-density and low-density datasets
mean_ranks_high = calculate_ranks(data_high)
mean_ranks_low = calculate_ranks(data_low)

# Plotting function to avoid repetition
def plot_ranked_absorbedPAR(mean_ranks, density_label, output_file):
    plt.figure(figsize=(10, 6))
    
    # Plotting lineplot for each architecture type within the current density
    for architecture in mean_ranks['architecture'].unique():
        architecture_data = mean_ranks[mean_ranks['architecture'] == architecture]
        sns.lineplot(x='rank', y='mean_rank', data=architecture_data, label=architecture, marker='o')
    
    plt.title(f'Ranked Absorbed PAR over Leaf Ranks for {density_label} Density')
    plt.xlabel('Leaf Rank')
    plt.ylabel('Mean Rank of Absorbed PAR')
    
    # Automatically adjust the y-axis based on data range
    plt.ylim(mean_ranks['mean_rank'].min(), mean_ranks['mean_rank'].max())
    plt.xticks(range(1, 11))
    plt.legend(title='Architecture Type')
    plt.savefig(output_file)
    plt.close()

# Plot for high-density data
plot_ranked_absorbedPAR(mean_ranks_high, 'High', 'plots/ranked_absorbedPAR_for_high_density.png')

# Plot for low-density data
plot_ranked_absorbedPAR(mean_ranks_low, 'Low', 'plots/ranked_absorbedPAR_for_low_density.png')




In [ ]:
'''PART 3: BAR PLOT OF TOTAL ABS PAR FOR PLANTS AND SENSORS'''

import pandas as pd
import matplotlib.pyplot as plt

# Load datasets for high and low densities
data_high_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_high_cleaned.csv')
data_low_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_low_cleaned.csv')
data_high_sensors = pd.read_csv('combined_files/combined_high_sensors_cleaned.csv')
data_low_sensors = pd.read_csv('combined_files/combined_low_sensors_cleaned.csv')


# Function to calculate mean, std dev, and confidence intervals
def calculate_statistics(data):
    grouped = data.groupby(['architecture'])['absorbedPAR_umol_m2_s1']
    mean_values = grouped.mean().reset_index(name='mean')
    mean_values['std_dev'] = grouped.std().reset_index(name='std_dev')['std_dev']
    mean_values['n'] = grouped.count().reset_index(name='count')['count']
        
    return mean_values

# Calculate statistics for both high-density and low-density datasets
mean_values_high_totalPAR_plants = calculate_statistics(data_high_totalPAR_plants)
mean_values_low_totalPAR_plants = calculate_statistics(data_low_totalPAR_plants)
mean_values_high_sensors = calculate_statistics(data_high_sensors)
mean_values_low_sensors = calculate_statistics(data_low_sensors)

# Plotting function to avoid repetition
def barplot_absorbedPAR(mean_values, density_label, output_file):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plotting lineplot for each architecture type within the current density
    for architecture in mean_values['architecture'].unique():
        architecture_data = mean_values[mean_values['architecture'] == architecture]
        ax.bar(architecture_data['architecture'], architecture_data['mean'], yerr=architecture_data['std_dev'], 
        width=0.5, edgecolor='black', linewidth=0.5, capsize=5, color='green')

    # Set the title and labels
    ax.set_title(f'Total Absorbed PAR for {density_label} density')
    ax.set_ylabel('Absorbed PAR (umol m-2 s-1)')
    plt.xticks(fontsize=5)

    plt.savefig(output_file)
    plt.close(fig)


# Call the barplot function.
barplot_absorbedPAR(mean_values_high_totalPAR_plants, 'High', 'plots/total_absorbedPAR_high_density.png')
barplot_absorbedPAR(mean_values_low_totalPAR_plants, 'Low', 'plots/total_absorbedPAR_low_density.png')
barplot_absorbedPAR(mean_values_high_sensors, 'High', 'plots/sensors_high_density.png')
barplot_absorbedPAR(mean_values_low_sensors, 'Low', 'plots/sensors_low_density.png')

print("Plots generated successfully for high and low densities.")